# Q&A over package.json with LangChain

In [2]:
import json
from pathlib import Path
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.indexes import VectorstoreIndexCreator
from langchain.docstore.document import Document


In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [49]:
file_path = "package_json_sample.json"
data = json.loads(Path(file_path).read_text())
documents = []
for i in data:
    repo_name = i["repo_name"]
    package_json_raw = i["package_json"]
    try:
        package_json = json.loads(package_json_raw)
        if type(package_json) is dict and "dependencies" in package_json:
            dependencies = package_json["dependencies"]
            page_content = (
                "dependencies=" + str(dependencies) + "\n" + "Repository=" + repo_name
            )
            # TODO figure out metadata
            d = Document(page_content=page_content, metadata={"Repository": repo_name})
            documents.append(d)
    except ValueError as e:
        print(f"Decoding JSON has failed for {repo_name}: {e}")

Decoding JSON has failed for pivotal-cf/dr-frankenstyle: Expecting value: line 1 column 1 (char 0)
Decoding JSON has failed for jonsamwell/blog_examples: Expecting property name enclosed in double quotes: line 11 column 1 (char 338)
Decoding JSON has failed for bradyhouse/house: Unexpected UTF-8 BOM (decode using utf-8-sig): line 1 column 1 (char 0)
Decoding JSON has failed for Self-Driving-Vehicle/kinect-butler: Expecting ',' delimiter: line 11 column 5 (char 299)
Decoding JSON has failed for bkirby989/WallaceTheme: Expecting value: line 7 column 19 (char 235)


In [50]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_documents(documents)

In [51]:
llm = ChatOpenAI(temperature = 0.0)
# query ="What are the dependencies for the Repository CiroArtigot/ababool"
query ="Which repositories have 'babel-runtime' as a dependency and in which version?"
# TODO
# query ="What is the most used dependency?"
response = index.query(query, llm, "", verbose=True)
display(Markdown(response))



> Entering new RetrievalQA chain...

> Finished chain.


The repositories that have 'babel-runtime' as a dependency and their respective versions are:

1. Repository: Ramshackle-Jamathon/Interactive-Experiments
   Dependency: babel-runtime
   Version: ^5.0.0

2. Repository: princesadie/SnapDrop-react
   Dependency: babel-runtime
   Version: ^5.0.0

3. Repository: aintnorest/ubiquitous-spoon
   Dependency: babel-runtime
   Version: ^6.0.0

4. Repository: SupremeTechnopriest/react-blueprint
   Dependency: babel-runtime
   Version: ^6.3.19